## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Faanui,PF,-16.4833,-151.7500,81.79,72,6,17.18,clear sky
1,1,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,26,3.00,scattered clouds
2,2,Yellowknife,CA,62.4560,-114.3525,-11.00,70,75,5.75,light snow
3,3,Cabo San Lucas,MX,22.8909,-109.9124,64.00,60,100,5.99,overcast clouds
4,4,Merauke,ID,-8.4667,140.3333,84.25,74,97,14.25,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Faanui,PF,-16.4833,-151.7500,81.79,72,6,17.18,clear sky
1,1,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,26,3.00,scattered clouds
4,4,Merauke,ID,-8.4667,140.3333,84.25,74,97,14.25,overcast clouds
5,5,Bluff,NZ,-46.6000,168.3333,77.00,49,100,11.99,overcast clouds
8,8,Mount Isa,AU,-20.7333,139.5000,89.60,48,0,5.75,clear sky
9,9,Avarua,CK,-21.2078,-159.7750,86.00,66,20,16.11,few clouds
11,11,Rikitea,PF,-23.1203,-134.9692,80.44,72,6,11.12,clear sky
12,12,Mataura,NZ,-46.1927,168.8643,77.00,49,100,11.99,overcast clouds
13,13,Port Macquarie,AU,-31.4333,152.9167,87.01,62,4,4.00,clear sky
15,15,Vaini,TO,-21.2000,-175.2000,82.40,83,75,11.50,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                169
City                   169
Country                169
Lat                    169
Lng                    169
Max Temp               169
Humidity               169
Cloudiness             169
Wind Speed             169
Current Description    169
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
between_df = preferred_cities_df.copy()
clean_df = between_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Faanui,PF,81.79,clear sky,-16.4833,-151.7500,
1,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,
4,Merauke,ID,84.25,overcast clouds,-8.4667,140.3333,
5,Bluff,NZ,77.00,overcast clouds,-46.6000,168.3333,
8,Mount Isa,AU,89.60,clear sky,-20.7333,139.5000,
9,Avarua,CK,86.00,few clouds,-21.2078,-159.7750,
11,Rikitea,PF,80.44,clear sky,-23.1203,-134.9692,
12,Mataura,NZ,77.00,overcast clouds,-46.1927,168.8643,
13,Port Macquarie,AU,87.01,clear sky,-31.4333,152.9167,
15,Vaini,TO,82.40,broken clouds,-21.2000,-175.2000,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [14]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Faanui,PF,81.79,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
1,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Castillo Galapagos
4,Merauke,ID,84.25,overcast clouds,-8.4667,140.3333,
5,Bluff,NZ,77.00,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
8,Mount Isa,AU,89.60,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
...,...,...,...,...,...,...,...
681,Mahibadhoo,MV,80.83,broken clouds,3.7833,72.9667,Omadhoo Inn
685,Tutoia,BR,79.77,overcast clouds,-2.7619,-42.2744,Pousada Guará's
687,Reyes,BO,75.60,overcast clouds,-14.3167,-67.3833,
689,Puerto Quijarro,BO,80.62,overcast clouds,-17.7833,-57.7667,Hotel palance


In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("",np.nan,inplace=True)
clean_hotel_df = hotel_df.dropna()

In [20]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Faanui,PF,81.79,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
1,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Castillo Galapagos
5,Bluff,NZ,77.00,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
8,Mount Isa,AU,89.60,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
9,Avarua,CK,86.00,few clouds,-21.2078,-159.7750,Paradise Inn
11,Rikitea,PF,80.44,clear sky,-23.1203,-134.9692,Pension Maro'i
12,Mataura,NZ,77.00,overcast clouds,-46.1927,168.8643,Ellie's Villa
13,Port Macquarie,AU,87.01,clear sky,-31.4333,152.9167,Rydges Port Macquarie
15,Vaini,TO,82.40,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
20,Isangel,VU,86.00,broken clouds,-19.5500,169.2667,Volcano Sea View Tree House


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))